In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle

import riiideducation

Вечер в хату, арестанты

In [ ]:
types = {
        'row_id': 'int64', 
        'timestamp': 'int64', 
        'user_id': 'int32', 
        'content_id': 'int16', 
        'content_type_id': 'int8',
        'task_container_id': 'int16', 
        'user_answer': 'int8', 
        'answered_correctly': 'int64', 
        'prior_question_elapsed_time': 'float32', 
        'prior_question_had_explanation': 'boolean'
}

In [ ]:
train = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv', 
    low_memory=False, 
    nrows=10**7, 
    dtype=types
)
questions=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                      dtype={'question_id': 'int16','part': 'int8','bundle_id': 'int16','tags': 'str'})
lectures=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv',
                    dtype={'lectures_id': 'int16','tag': 'int16','part': 'int8','type_of': 'str'})
ex_test=pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

train.head()

**train**

row_id: (int64) ID code for the row.

timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

user_id: (int32) ID code for the user.

content_id: (int16) ID code for the user interaction

content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.

prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.

In [ ]:
questions.tail()

**QUESTIONS.CSV**

question_id = content_id: foreign key for the train/test content_id column, when the content type is question (0).

bundle_id: code for which questions are served together.

correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

part: top level category code for the question.

tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [ ]:
questions.iloc[5716]

In [ ]:
lectures.head(10)

lectures.csv: metadata for the lectures watched by users as they progress in their education.

lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).

part: top level category code for the lecture.

tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

type_of: brief description of the core purpose of the lecture

In [ ]:
train.head()

In [ ]:
ex_test.shape

In [ ]:
ex_test.head(50)

example_test_rows.csv Three sample groups of the test set data as it will be delivered by the time-series API. The format is largely the same as train.csv. There are two different columns that mirror what information the AI tutor actually has available at any given time, but with the user interactions grouped together for the sake of API performance rather than strictly showing information for a single user at a time. Some users will appear in the hidden test set that have NOT been presented in the train set, emulating the challenge of quickly adapting to modeling new arrivals to a website.

prior_group_responses (string) provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.

prior_group_answers_correct (string) provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.

**Какие признаки нам помогут для предсказаний?**

1) запоминаем историю ответов пользователя ( долю правильных ответов конкретного пользователя = accuracy)

2) запоминаем историю овтетов всех пользователей на конкретные вопросы( долю правильных ответов на конкретный вопрос)

????3) запоминаем историю ответов для блоков вопросов ( но нужно ли? мб 2 уже включает это)

4) смотеть, просмотрены ли лекции по материалам вопроса( через скалярное произведение tag question на tag просмотренных лекций, делёное на кол-во tag вопроса)

5??5-6) запоминаем среднее время на правильный и/или неправильный ответ для конкретного пользователя(prior_question_elapsed_time делаем lag -1) или запоминаем просто среднее время
или только ср вр на правильный ответ

7??7-8) запоминаем среднее время на правильный и/или неправильный ответ для конкретного вопроса  
или запоминаем просто среднее время или только ср вр на правильный ответ

9+) запоминаем accuracy ответов для каждой из групп вопросов, полученных после класстеризации вопросов по tag questions

10+)  запоминаем среднее время ответов для каждой из групп вопросов, полученных после класстеризации вопросов по tag questions

In [ ]:
#env = riiideducation.make_env()
#iter_test = env.iter_test()
#for (test_df, sample_prediction_df) in iter_test:
    # merge
#    print(test_df)

In [ ]:
#test_df.shape

In [ ]:
#test_df.head(200)

In [ ]:
train

## Стоит обратить внимание на колонки prior_group_answers_correct	prior_group_responses, стоит  почитаь о них в discussion

## FEATURE ENGINEERING and DATA CLEANING

## Пока что работаем без валидационной выборки, потом добавил, если понадобится

In [ ]:
# разделяем на обучающий и валидационный наборы

In [ ]:
train_questions_only_df = train[train['answered_correctly']!=-1]
grouped_by_user_df = train_questions_only_df.groupby('user_id')


user_ans_df = grouped_by_user_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count']}
).copy()

user_ans_df.columns = [
    'mean_user_accuracy',
    'questions_answered'
]

In [ ]:
user_ans_df

In [ ]:
## пока что просто среднее время на ответ без разграничения на правильность
user_ans_time_df = grouped_by_user_df.agg(
    {
        'prior_question_elapsed_time': [
            'mean']}
).copy()

user_ans_time_df.columns = [
    'mean_user_ans_time'
]

user_ans_time_df

In [ ]:
f

In [ ]:
grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg(
    {
        'answered_correctly': [
            'mean', 
            'count'
        ]
    }
).copy()

content_answers_df.columns = [
    'mean_que_accuracy', 
    'question_was_asked'
]

content_answers_df

In [ ]:
f = []
for i in range(len(train)-1):
    f.append(train['prior_question_elapsed_time'].values[i+1])
f.append(0)
train['prior_question_elapsed_time']= f
train

In [ ]:
train_questions_only_well = train[train['answered_correctly']!=0]

In [ ]:
grouped_by_questions_well = train_questions_only_well.groupby('content_id')
quest_well_df = grouped_by_questions_well.agg(
    {
    'prior_question_elapsed_time':['mean']
    }).copy()
quest_well_df.columns = [

    'mean_prior_question_elapsed_time']

quest_well_df["mean_prior_question_elapsed_time"] = quest_well_df["mean_prior_question_elapsed_time"].astype(int)


quest_well_df

In [ ]:
grouped_by_user_well = train_questions_only_well.groupby('user_id')
user_well_df = grouped_by_user_well.agg(
    {
    'prior_question_elapsed_time':['mean']
    }).copy()
user_well_df.columns = [

    'mean_prior_question_elapsed_time_user']
user_well_df.fillna(max)
#user_well_df["mean_prior_question_elapsed_time_user"] = user_well_df["mean_prior_question_elapsed_time_user"].astype(int)


user_well_df

## Получили фичи 1-2, 5-8. Далее можно улучшить 5-8 и заняться кластеризацией для фичей 4 и 9+

In [ ]:
#questions
#lectures
questions

In [ ]:
import re
import sys

In [ ]:
s = set()
#dirt_list = questions["tags"]
dirt_list = questions["tags"].str.split(" ", expand = True) 
for merge in dirt_list:
    ns = set(dirt_list[merge])
    s.update(ns)
   

s

In [ ]:
s = {i for i in s if pd.notna(i)}
s

In [ ]:
tag_df = pd.DataFrame(index=questions.value_counts(), columns=s)
tag_df = tag_df.fillna(0)

tag_df.index = [i for i in range(13522)]
tag_df

In [ ]:
buf = []
tagg_df = pd.DataFrame(index=questions.value_counts())
tagg_df.index = [i for i in range(13522)]

f = questions['tags'].dropna()
for i in s:
    buf = []
    for j in range(len(f.values)):
        d = f.values[j].split(' ')
        #print(d)
        if i in d:
            buf.append(1)
        else:
            buf.append(0)
    df = pd.DataFrame(buf, columns = [i])
    #print(df)
    tagg_df = tagg_df.join(df)
#tagg_df

In [ ]:
tagg_df

## Фича 4 (связь вопросов с лекциями)

In [ ]:
grouped_by_container_df = train.groupby('task_container_id')
container_df = grouped_by_container_df.agg(
    {'answered_correctly': [
            'mean', 
            'count'],
    'content_type_id':['sum']
    }).copy()
container_df.columns = [
    'mean_container_acc', 
    'container_unit_count',
    'lecs_in_container']
container_df["lecs_in_container"] = container_df["lecs_in_container"].astype(int)
container_df["lecs_to_qs"] = container_df["lecs_in_container"]/container_df["container_unit_count"]

container_df

In [ ]:
target = 'answered_correctly'
train = train[train[target] != -1] #delet all lections
#merge all features in one df
train = train.merge(user_ans_df, how='left', on='user_id')
train = train.merge(content_answers_df, how='left', on='content_id')
train = train.merge(container_df, how='left', on='task_container_id')
#train = train.merge(user_ans_time_df, how='left', on='user_id')
#train = train.merge(que_ans_time_df, how='left', on='content_id')
train = train.merge(quest_well_df, how='left', on='content_id')
train = train.merge(user_well_df, how='left', on='user_id')

train

## Получили набор данных, теперь нужно его почистить от пропусков, выбросов, удалить лишнии колонки

In [ ]:
train.isnull().sum()

In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(value=False).astype(bool)
train = train.fillna(train.mean())

In [ ]:
col_to_drop = ['timestamp','row_id','user_id','content_id','content_type_id','task_container_id', 'user_answer']
train=train.drop(columns = col_to_drop)

train

In [ ]:
features = ['prior_question_elapsed_time',
       'prior_question_had_explanation', 'mean_user_accuracy',
       'questions_answered', 'mean_que_accuracy', 'question_was_asked',
       'mean_container_acc', 'container_unit_count', 'lecs_in_container',
       'lecs_to_qs', 'mean_prior_question_elapsed_time',
       'mean_prior_question_elapsed_time_user']

features

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(train[features], train[target], random_state=300, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 300, max_depth=5, criterion = "entropy",n_jobs = -1)
model.fit(train_df, y_train)
print('Forest ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1]))

In [ ]:
## SUBMISSION
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    
    test_df = test_df.merge(user_ans_df, how='left', on='user_id')
    test_df = test_df.merge(content_answers_df, how='left', on='content_id')
    test_df = test_df.merge(container_df, how='left', on='task_container_id')
    #test_df = test_df.merge(user_ans_time_df, how='left', on='user_id')
    #test_df = test_df.merge(que_ans_time_df, how='left', on='content_id')
    train = train.merge(quest_well_df, how='left', on='content_id')
    train = train.merge(user_well_df, how='left', on='user_id')
    
    # type transformation
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(df.mean(), inplace = True)
    
    # preds
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])